## 2.3 Spark SQL

Spark SQL is a layer on top of Spark core that allows for querying large datasets using a SQL-like language.

**Warning:** In your production cluster, you have Spark 1.2, which is what we've also installed on your laptops.  You should know that Spark SQL is one of the components that has evolved the most since Spark 1.2 to the current Spark 1.5.  In particular, the internal engine that translates SQL to Spark operations is much better in later versions of Spark, and a DataFrame API similar to Pandas has been added.

**Warning 2:** In Spark 1.2, there is a [bug in the Spark SQL parser](https://issues.apache.org/jira/browse/SPARK-5009) that causes a stack overflow error in Windows (though curiously, not on Macs or on Linux machines).  If you trigger this bug, please use Spark 1.3 or later to follow along.

Most of what you learn today will carry over into later versions of Spark.  If you do nothing, you'll at least see substantial performance improvements.  When you *do* upgrade, invest some time in learning the new features that Spark SQL has acquired.

First things first: to use Spark SQL, we need to set up a `SQLContext` that wraps around the existing `SparkContext` and adds extra functionality.

In [1]:
from pyspark import SQLContext, Row
sqlCtx = SQLContext(sc)

Spark SQL adds a new kind of RDD called a **`SchemaRDD`**, essentially a table.  Each item in a schema RDD is a `Row`, and there's an overall schema to the RDD as well.

The simplest way to create a `SchemaRDD` is with `sc.parallelize`.  Here's an example:

In [2]:
rdd = sc.parallelize([
        Row(name='John', home='Brussels', age=35),
        Row(name='Jack', home='Brussels', age=32),
        Row(name='Jane', home='Leuven', age=42),
        Row(name='Jill', home='Mechelen', age=53),
    ])

Once we have an RDD of `Row`s, we ask Spark SQL to infer the schema and expose the RDD as a SQL table.

In [3]:
schemaRDD = sqlCtx.inferSchema(rdd)
schemaRDD.registerTempTable("people")

**Note:** Tomorrow, we'll discuss exposing Hive tables in Spark SQL, which will likely be the main way to connect to your production data.  Spark SQL can also expose lots of other tables stored in common formats (JSON, ORC, Parquet, ...).  See [Learning Spark](http://shop.oreilly.com/product/0636920028512.do) for details.

We can now run SQL-like queries over this dataset:

In [4]:
avgAge = sqlCtx.sql(
    """SELECT
           home,
           AVG(age) AS mean
       FROM people
       GROUP BY home"""
)
avgAge

MapPartitionsRDD[16] at mapPartitions at SerDeUtil.scala:143

As in SQL, the results of a query are themselves a table.  In Spark SQL, the results are a kind of SchemaRDD, namely, an RDD with `Row`s for items.

In [5]:
avgAge.collect()

[Row(home=u'Brussels', mean=33.5),
 Row(home=u'Mechelen', mean=53.0),
 Row(home=u'Leuven', mean=42.0)]

Being an RDD, you can apply the normal Spark operations to the results.  Fyi, you can access the fields inside `Row` objects using the notation `row.fieldname`.  Here's an example:

In [6]:
print(avgAge
      .map(lambda row: "Average age in {0} is {1} years"
                        .format(row.home, row.mean))
      .reduce(lambda x, y: x + "\n" + y))

Average age in Brussels is 33.5 years
Average age in Mechelen is 53.0 years
Average age in Leuven is 42.0 years


**Ex 2.3.1 Earlier, we had set up a Pandas DataFrame of GDP per capita for Belgium and the Netherlands in the years 1913, 1950 and 2003 as follows:**
```
data = {
    'country': ['BE', 'BE', 'BE', 'NL', 'NL', 'NL'],
    'year': [1913, 1950, 2003, 1913, 1950, 2003],
    'gdp_per_capita': [4220, 5462, 21205, 4049, 5996, 21480]
}
frame = DataFrame(data)
```
**Expose an equivalent table in Spark SQL.  What has been the average GDP per capita of these two countries in the years concerned?**

The result of a `sqlCtx.sql("...")` statement is itself a `SchemaRDD`, so can be registered as a new temporary table in Spark SQL.  In this way, you can build up queries in steps, much like in stored procedures or Common Table Expressions.

**Ex 2.3.2 Using queries to extract the Belgian and Dutch datasets, then joining by year, calculate the percent difference in GDP per capita between BE and NL in all years concerned.**

---

Let's work through an extended example of exposing the `names` dataset from before as a `SchemaRDD` and using Spark SQL to massage the data therein.

**Ex 2.3.3  Using `sc.textFile`, `union` and `map`, expose the `names` dataset as a SchemaRDD with columns `year`, `name`, `sex` and `births`.  Register it as a temp table called `"names"`**

The following exercises should be straightforward given your existing SQL skills. 

**Ex 2.3.4  How many births took place in the US from 1939 to 1945?**  
_Note:_ the answer isn't quite accurate, because the underlying `names` dataset only includes names for which at least 5 births took place in any given year.  But let's ignore this detail.

**Ex 2.3.5 Make a plot of the number of babies called `"Mary"` (either gender) born each year from 1880 to 2014**

**Ex 2.3.6 Make a plot of the number of boys and girls born each year from 1880 to 2014**

Note that Spark SQL didn't introduce windowing function (e.g., `ROW_NUMBER() OVER (PARTITION BY ...)`) until Spark 1.4.  So the following exercise can't be done with just a simple SQL query.  However, what we can do is mix Spark SQL with Spark core whenever something is hard to express in SQL.

**(!) Ex 2.3.7  Make a plot of the share of births represented by the top 1000 names each year**

As in previous sessions, we've just scratched the surface of Spark SQL, but I hope you get a sense of what's possible.  Many other basic SQL features work as you expect:
* Complex Joins
* Subqueries
* Basic math functions
* User-defined functions (UDFs)
* ...

See the full list [here](https://spark.apache.org/docs/1.2.0/sql-programming-guide.html#compatibility-with-apache-hive).  Every Spark version adds more SQL features, so keep an eye out for when your big data team upgrades Spark on your cluster.  For instance, four important features missing in Spark 1.2 but present in Spark 1.5 are:
* Exposing tables in existing RDBMSs (e.g., Oracle, Postgres, MySQL)
* Common Table Expression (`WITH temptable AS (SELECT ...) SELECT ... FROM temptable`)
* Windowing functions (`ROW_NUMBER() OVER (PARTITION BY ...)`)
* Lots more predefined functions